# RAG Using LangChain Mistral and Weaviate(Cloud) DB

Using Opensource libraries

In [ ]:
# Import general libraries
import os
from dotenv import load_dotenv

In [5]:
# Load environment variables
load_dotenv()

True

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings

In [7]:
# Define Embedding
embeddings = HuggingFaceEmbeddings(model="sentence-transformers/all-mpnet-base-v2")